HW4 - NLP

Task : Products

Section : Text Generation

Author : Hamidreza Amirzadeh 401206999

Group 3



In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install bert_score

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

import pandas as pd
import random
import math
import time

# Load dataset and preprocessing

In [3]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_excel('./drive/MyDrive/keifiat.xlsx')

Mounted at /content/drive


In [4]:
df.head()

,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید._x000D_\nیه...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"


In [5]:
df_new = df[['title', 'comment']]

df_new.head()

,title,comment
0,NaN,واقعا عالیه. من که ازش خیلی راضیم
1,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
2,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...
3,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...
4,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید._x000D_\nیه...


In [6]:
df_new.shape

(100000, 2)

Duo to high amount of data and long time need for training, we choose 10000 examples randomly from dataset

In [7]:
df_new = df_new.sample(10000, random_state=42)
df_new = df_new[(df_new['title'].isnull() | df_new['comment'].isnull()) == False]
df_new.shape

(9673, 2)

In [8]:
from sklearn.model_selection import train_test_split

df_new = df_new.astype(str)
df_train, df_test = train_test_split(df_new, test_size=0.1, random_state=1)
df_train, df_val = train_test_split(df_train, train_size=0.8, random_state=1)

In [9]:
df_train.head()

,title,comment
65099,خوبه اما این قیمت نمیارزه!!!,۱- از موبوتک خریدم داخل بسته به جز گلس چیز دیگ...
95799,از همه نظر عالیه,الان به دستم رسید واقعا کیفیت عالی داره حتما پ...
98735,گزینه خوبی برای چراغ عقب,بهترین گزینه برای چراغ عقب هست. _x000D_\nاگر ش...
78803,اندازه کوچک,از نمونه تو بازارهم کوچکتره و هم گرون تر ،بخاط...
96928,حتما بخرید,شکیل و زیباست پا در این کفش بو نمیده


In [10]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

In [11]:
from datasets.dataset_dict import DatasetDict

digikala_dataset = DatasetDict({'train':train_dataset, 'val':val_dataset, 'test':test_dataset})
digikala_dataset = digikala_dataset.remove_columns('__index_level_0__')
digikala_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'comment'],
        num_rows: 6964
    })
    val: Dataset({
        features: ['title', 'comment'],
        num_rows: 1741
    })
    test: Dataset({
        features: ['title', 'comment'],
        num_rows: 968
    })
})

In [12]:
def show_samples(dataset, num_samples=3, seed=28):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['title']}'")
        print(f"'>> Comment: {example['comment']}'")


show_samples(digikala_dataset)


'>> Title: ملیحه ضیایی'
'>> Comment: سلام واقعا ریمل بیخودی بود من دوبار استفاده کردم و انداختم سطل زباله با این قیمت. اصلا ارزش خرید ندارد'

'>> Title: ساعت مچی'
'>> Comment: از هر نظر عالی وشیک این برند ژاپنی در آینده محصولاتش گرانتر خواهد شد'

'>> Title: پیشنهاد نمیکنم'
'>> Comment: اصلا پیشنهاد نمیکنم؛ بخاطر اینکه وزن 1.11 گرمی آن  نسبت به قیمت آن همخوانی ندارد.'


In [13]:
digikala_dataset = digikala_dataset.filter(lambda x: len(x["title"].split()) > 2)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
digikala_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'comment'],
        num_rows: 3388
    })
    val: Dataset({
        features: ['title', 'comment'],
        num_rows: 848
    })
    test: Dataset({
        features: ['title', 'comment'],
        num_rows: 482
    })
})

# Choose tokenizer and model

In [15]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [16]:
inputs = tokenizer("به نام خداوند جان و خرد")
inputs

{'input_ids': [554, 5738, 16623, 15638, 15267, 341, 13906, 883, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁به', '▁نام', '▁خدا', 'وند', '▁جان', '▁و', '▁خر', 'د', '</s>']

In [18]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(examples["comment"], max_length=max_input_length, truncation=True)
    labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

 tokenize the whole corpus:

In [19]:
tokenized_datasets = digikala_dataset.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'comment', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3388
    })
    val: Dataset({
        features: ['title', 'comment', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 848
    })
    test: Dataset({
        features: ['title', 'comment', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 482
    })
})

In [21]:
tokenized_datasets['train'][0]

{'title': 'خوبه اما این قیمت نمیارزه!!!',
 'comment': '۱- از موبوتک خریدم داخل بسته به جز گلس چیز دیگه ای نبود و برای تمیز کردن صفحه گوشی قبل از نصب مجبور به استفاده از شیشه پاک کن و دستمال دیگری شدم_x000D_\n۲- کادرش یه کوچولو از صفحه گوشی کوچیکتره و نوشته های کناری راست و چپ میره زیر کادر گلس البته نه خیلی زیاد_x000D_\n۳- تاچ رو ضعیف نکرده و شفافیتش خوبه_x000D_\n۴- یه کم صفحه رو تاریک میکنه که مهم نیست و چشم عادت میکنه_x000D_\n۵- نصبش راحت انجام شد بدون اینکه حباب زیر گلس بمونه',
 'input_ids': [25464,
  264,
  695,
  3433,
  43562,
  1505,
  8579,
  633,
  10760,
  614,
  7088,
  554,
  16142,
  12176,
  1471,
  1177,
  9776,
  118246,
  376,
  1997,
  259,
  586,
  25274,
  341,
  259,
  1699,
  5323,
  9776,
  14473,
  259,
  19691,
  259,
  21690,
  406,
  5127,
  695,
  14165,
  1050,
  4952,
  16742,
  554,
  259,
  6691,
  695,
  2239,
  52494,
  36388,
  5692,
  341,
  5311,
  22146,
  7146,
  406,
  3164,
  633,
  290,
  329,
  10874,
  403,
  290,
  26055,
  264,
  1072,
  56

##  Metrics we want to use

In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")
bert_score = evaluate.load("bertscore")
bleu_score = evaluate.load('bleu')

In [23]:
generated_title = "گوشی بسیار خوبی هست و شارژ زیادی نگه میداره"
reference_title = "من از خرید این گوشی خوب که شارژ دهی زیادی داره راضی هستم"

In [24]:
scores = rouge_score.compute(predictions=[str(tokenizer(generated_title)['input_ids'])],
                             references=[str(tokenizer(reference_title)['input_ids'])])
scores

{'rouge1': 0.6190476190476191,
 'rouge2': 0.3,
 'rougeL': 0.5714285714285715,
 'rougeLsum': 0.5714285714285715}

In [46]:
scores = bert_score.compute(predictions=[generated_title],
                            references=[reference_title],
                            lang='fa')
scores

{'precision': [0.7659614682197571],
 'recall': [0.744793176651001],
 'f1': [0.7552289962768555],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.26.0)'}

In [47]:
scores = bleu_score.compute(predictions=[str(tokenizer(generated_title)['input_ids'])],
                             references=[str(tokenizer(reference_title)['input_ids'])])
scores

{'bleu': 0.43687342763905745,
 'precisions': [0.8461538461538461,
  0.6578947368421053,
  0.4864864864864865,
  0.3055555555555556],
 'brevity_penalty': 0.8145428811424836,
 'length_ratio': 0.8297872340425532,
 'translation_length': 39,
 'reference_length': 47}

## Fine-tuning mT5

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 4
num_train_epochs = 7

logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-digikala-titleGen",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    # we should generate titles during evaluation so that we can compute ROUGE scores for each epoch
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

provide the trainer with a compute_metrics() function so that we can evaluate our model during training

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Compute ROUGE scores
    result = rouge_score.compute(predictions=[str(tokenizer(decoded_preds)['input_ids'])], 
                                 references=[str(tokenizer(decoded_labels)['input_ids'])], use_stemmer=True)
    # Extract the scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

define a data collator for our sequence-to-sequence task:

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# remove the columns with strings because the collator won’t know how to pad these elements:
tokenized_datasets = tokenized_datasets.remove_columns(digikala_dataset['train'].column_names)

Since the collator expects a list of dicts, where each dict represents a single example in the dataset, we also need to wrangle the data into the expected format before passing it to the data collator:

In [ ]:
features = [tokenized_datasets["train"][i] for i in range(2)]
features

[{'input_ids': [25464,
   264,
   695,
   3433,
   43562,
   1505,
   8579,
   633,
   10760,
   614,
   7088,
   554,
   16142,
   12176,
   1471,
   1177,
   9776,
   118246,
   376,
   1997,
   259,
   586,
   25274,
   341,
   259,
   1699,
   5323,
   9776,
   14473,
   259,
   19691,
   259,
   21690,
   406,
   5127,
   695,
   14165,
   1050,
   4952,
   16742,
   554,
   259,
   6691,
   695,
   2239,
   52494,
   36388,
   5692,
   341,
   5311,
   22146,
   7146,
   406,
   3164,
   633,
   290,
   329,
   10874,
   403,
   290,
   26055,
   264,
   1072,
   5658,
   1440,
   259,
   2632,
   57342,
   15878,
   695,
   259,
   19691,
   259,
   21690,
   406,
   57342,
   11804,
   72380,
   341,
   24491,
   376,
   1091,
   5692,
   13182,
   41485,
   341,
   1177,
   5473,
   822,
   5858,
   12578,
   1072,
   5658,
   12176,
   1471,
   402,
   30394,
   3237,
   1804,
   10404,
   8063,
   883,
   290,
   329,
   10874,
   403,
   290,
   31051,
   264,
   2301,
   3

In [ ]:
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[ 25464,    264,    695,   3433,  43562,   1505,   8579,    633,  10760,
            614,   7088,    554,  16142,  12176,   1471,   1177,   9776, 118246,
            376,   1997,    259,    586,  25274,    341,    259,   1699,   5323,
           9776,  14473,    259,  19691,    259,  21690,    406,   5127,    695,
          14165,   1050,   4952,  16742,    554,    259,   6691,    695,   2239,
          52494,  36388,   5692,    341,   5311,  22146,   7146,    406,   3164,
            633,    290,    329,  10874,    403,    290,  26055,    264,   1072,
           5658,   1440,    259,   2632,  57342,  15878,    695,    259,  19691,
            259,  21690,    406,  57342,  11804,  72380,    341,  24491,    376,
           1091,   5692,  13182,  41485,    341,   1177,   5473,    822,   5858,
          12578,   1072,   5658,  12176,   1471,    402,  30394,   3237,   1804,
          10404,   8063,    883,    290,    329,  10874,    403,    290,  31051,
            26

In [ ]:
data_collator(features).keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

The main thing to notice here is that the first example is longer than the second one, so the input_ids and attention_mask of the second example have been padded on the right with a [PAD] token (whose ID is 0). Similarly, we can see that the labels have been padded with -100s, to make sure the padding tokens are ignored by the loss function. And finally, we can see a new decoder_input_ids which has shifted the labels to the right by inserting a [PAD] token in the first entry.

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/Hamid-reza/mt5-small-finetuned-digikala-titleGen into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3388
  Num Epochs = 7
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 5929
  Number of trainable parameters = 300176768


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,7.555500,3.259381,45.672900,19.644600,31.597400,31.597400
2,4.138600,3.034716,58.302100,32.817200,33.901200,33.901200
3,3.744900,2.966547,66.731000,40.899100,34.220300,34.220300
4,3.557500,2.910164,65.598000,39.408100,34.511600,34.511600
5,3.406200,2.894409,69.608100,42.870700,34.662200,34.662200
6,3.340800,2.888751,70.212300,42.863900,34.566900,34.566900
7,3.302500,2.880115,70.348900,43.245000,34.660800,34.660800


Streaming output truncated to the last 5000 lines.
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {


TrainOutput(global_step=5929, training_loss=4.1494319746558235, metrics={'train_runtime': 3467.2274, 'train_samples_per_second': 6.84, 'train_steps_per_second': 1.71, 'total_flos': 3304574034862080.0, 'train_loss': 4.1494319746558235, 'epoch': 7.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 848
  Batch size = 4
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_

{'eval_loss': 2.880114793777466,
 'eval_rouge1': 70.3489,
 'eval_rouge2': 43.245,
 'eval_rougeL': 34.6608,
 'eval_rougeLsum': 34.6608,
 'eval_runtime': 125.2149,
 'eval_samples_per_second': 6.772,
 'eval_steps_per_second': 1.693,
 'epoch': 7.0}

In [ ]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

Saving model checkpoint to mt5-small-finetuned-digikala-titleGen
Configuration saved in mt5-small-finetuned-digikala-titleGen/config.json
Configuration saved in mt5-small-finetuned-digikala-titleGen/generation_config.json
Model weights saved in mt5-small-finetuned-digikala-titleGen/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-digikala-titleGen/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-digikala-titleGen/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-digikala-titleGen/spiece.model


Upload file pytorch_model.bin:   0%|          | 32.0k/1.12G [00:00<?, ?B/s]

Upload file runs/Feb07_19-23-03_9d62e0d4d3ba/events.out.tfevents.1675801432.9d62e0d4d3ba.531.2: 100%|#########…

Upload file runs/Feb07_19-23-03_9d62e0d4d3ba/events.out.tfevents.1675797797.9d62e0d4d3ba.531.0: 100%|#########…

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/Hamid-reza/mt5-small-finetuned-digikala-titleGen
   fb21be5..e6cee3a  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Hamid-reza/mt5-small-finetuned-digikala-titleGen
   fb21be5..e6cee3a  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 70.3489}]}
To https://huggingface.co/Hamid-reza/mt5-small-finetuned-digikala-titleGen
   e6cee3a..7f9fedf  main -> main

   e6cee3a..7f9fedf  main -> main



'https://huggingface.co/Hamid-reza/mt5-small-finetuned-digikala-titleGen/commit/e6cee3a73b67024e2e1c7fea00d6e2deb5a98983'

# Load our trained model and evaluate it

In [25]:
from transformers import pipeline

hub_model_id = "Hamid-reza/mt5-small-finetuned-digikala-titleGen"
summarizer = pipeline("summarization", model=hub_model_id)

In [26]:
def print_title(idx):
    comment = digikala_dataset["test"][idx]["comment"]
    title = digikala_dataset["test"][idx]["title"]
    gen_title = summarizer(digikala_dataset["test"][idx]["comment"])[0]["summary_text"]
    print(f"'>>> Comment: {comment}'")
    print(f"\n'>>> Title: {title}'")
    print(f"\n'>>> Generated Title: {gen_title}'")

In [27]:
print_title(45)

Your max_length is set to 20, but you input_length is only 11. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


'>>> Comment: فقط میتونم بگم عااااالی'

'>>> Title: فقط میتونم بگم عاااالی'

'>>> Generated Title: خیلی خوبه'


In [28]:
print_title(255)

'>>> Comment: من این دریل رو تو شگفت انگیز خریدم . اصلا قدرت سوراخ کاری ساده رو هم نداره _x000D_
یک کاشی رو به زور سوراخ میکنه . _x000D_
فقط قیمتش نسبت به خریدش اون موقع مناسب بود ولی الان نه .'

'>>> Title: نارضایتی از محصول'

'>>> Generated Title: اصلا مناسب نیست'


## Compute metrics for test set

In [33]:
df_test

,title,comment
28921,چند روزی میشه خریدم,حتما پیشنهاد میکنم
75109,کیفیت و قیمت,نمک‌خوبی هست
95790,کیفیت عالی و قیمت مناسب,من این لیف رو خریدم و چند بار استفاده کردم خیل...
57368,کیفیت پایین,از نظر مقاومت قطعات واقعا بد هستش،با کوچکترین ...
3282,گوشی عالیه,عاااااااللللیه هرچی بگم کم گفتم این گوشی عالیه...
...,...,...
45546,از همه لحاظ عالی,من خیلی تحقیق کرده بودم در مورد اسپیکر، صدای ف...
25269,میرزاخانی,خیلی خوشمزه ست هم طعم کروسان عالیه هم طعم کاکا...
81457,معمولی,کیفیت معمولی
61079,خرید کالای با کیفیت,کیفیت صدا و بیس بسیار عالی داره _x000D_\nاگر ...


In [ ]:
df_test['gen-title'] = df_test.apply(lambda x: summarizer(x["comment"])[0]["summary_text"], axis=1)

In [40]:
df_test

,title,comment,gen-title
28921,چند روزی میشه خریدم,حتما پیشنهاد میکنم,پیشنهاد میکنم
75109,کیفیت و قیمت,نمک‌خوبی هست,خیلی خوبه
95790,کیفیت عالی و قیمت مناسب,من این لیف رو خریدم و چند بار استفاده کردم خیل...,خیلی راضیم
57368,کیفیت پایین,از نظر مقاومت قطعات واقعا بد هستش،با کوچکترین ...,خیلی خوب هست
3282,گوشی عالیه,عاااااااللللیه هرچی بگم کم گفتم این گوشی عالیه...,خیلی عالیه
...,...,...,...
45546,از همه لحاظ عالی,من خیلی تحقیق کرده بودم در مورد اسپیکر، صدای ف...,خیلی عالی و عالی
25269,میرزاخانی,خیلی خوشمزه ست هم طعم کروسان عالیه هم طعم کاکا...,عالیه
81457,معمولی,کیفیت معمولی,کیفیت معمولی
61079,خرید کالای با کیفیت,کیفیت صدا و بیس بسیار عالی داره _x000D_\nاگر ...,خیلی عالی و با کیفیت


In [79]:
df_test['rouge1 score'] = df_test.apply(lambda x: rouge_score.compute(predictions=[str(tokenizer(x['gen-title'])['input_ids'])],
                                                                      references=[str(tokenizer(x['title'])['input_ids'])])['rouge1'],
                                                                      axis=1)

df_test['bert score'] = df_test.apply(lambda x: bert_score.compute(predictions=[x['gen-title']],
                                    references=[x['title']],
                                    lang='fa')['f1'][0],
                                    axis=1)

df_test['bleu score'] = df_test.apply(lambda x: bleu_score.compute(predictions=[str(tokenizer(x['gen-title'])['input_ids'])],
                                    references=[str(tokenizer(x['title'])['input_ids'])])['bleu'],
                                    axis=1)

In [80]:
df_test

,title,comment,gen-title,rouge1 score,bert score,bleu score
28921,چند روزی میشه خریدم,حتما پیشنهاد میکنم,پیشنهاد میکنم,0.307692,0.735245,0.228695
75109,کیفیت و قیمت,نمک‌خوبی هست,خیلی خوبه,0.181818,0.679599,0.000000
95790,کیفیت عالی و قیمت مناسب,من این لیف رو خریدم و چند بار استفاده کردم خیل...,خیلی راضیم,0.266667,0.709291,0.000000
57368,کیفیت پایین,از نظر مقاومت قطعات واقعا بد هستش،با کوچکترین ...,خیلی خوب هست,0.181818,0.730295,0.000000
3282,گوشی عالیه,عاااااااللللیه هرچی بگم کم گفتم این گوشی عالیه...,خیلی عالیه,0.615385,0.861223,0.584103
...,...,...,...,...,...,...
45546,از همه لحاظ عالی,من خیلی تحقیق کرده بودم در مورد اسپیکر، صدای ف...,خیلی عالی و عالی,0.500000,0.760022,0.457208
25269,میرزاخانی,خیلی خوشمزه ست هم طعم کروسان عالیه هم طعم کاکا...,عالیه,0.222222,0.820607,0.000000
81457,معمولی,کیفیت معمولی,کیفیت معمولی,0.800000,0.866117,0.576084
61079,خرید کالای با کیفیت,کیفیت صدا و بیس بسیار عالی داره _x000D_\nاگر ...,خیلی عالی و با کیفیت,0.500000,0.806260,0.456280


In [84]:
df_test[['rouge1 score', 'bert score', 'bleu score']].describe()

,rouge1 score,bert score,bleu score
count,968.000000,968.000000,968.000000
mean,0.324888,0.733212,0.088294
std,0.173364,0.063618,0.181495
min,0.043478,0.501041,0.000000
25%,0.190476,0.692851,0.000000
50%,0.285714,0.725703,0.000000
75%,0.400000,0.761652,0.000047
max,1.000000,1.000000,1.000000
